# 🚀 Mindneox.ai - Free GPU Testing on Google Colab

**No installation needed on Mac! Run everything in your browser with FREE GPU.**

## Setup Instructions:
1. Click **Runtime** → **Change runtime type** → **GPU** → **Save**
2. Run each cell in order (Shift + Enter)
3. Upload your files when prompted
4. Test with free T4 GPU (10x faster than Mac!)

---

## Step 1: Check GPU Availability

In [ ]:
import torch
import subprocess

print("=" * 60)
print("🔍 GPU Status Check")
print("=" * 60)

# Check CUDA
print(f"\n✅ CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"✅ CUDA Version: {torch.version.cuda}")
    print("\n🎉 FREE GPU IS READY!")
else:
    print("\n❌ GPU NOT ENABLED!")
    print("💡 Enable GPU: Runtime → Change runtime type → GPU → Save")
    print("Then restart this notebook.")

print("=" * 60)

## Step 2: Install Dependencies (Takes ~3 minutes)

In [ ]:
print("📦 Installing dependencies...")
print("⏱️  This will take about 3 minutes\n")

# Install PyTorch with CUDA (already included in Colab)
!pip install -q llama-cpp-python langchain langchain-core langchain-community
!pip install -q redis pinecone-client sentence-transformers
!pip install -q transformers accelerate

print("\n✅ All dependencies installed!")
print("✅ Ready for GPU acceleration!")

## Step 3: Download Mistral-7B Model (Takes ~2 minutes)

In [ ]:
import os
from pathlib import Path

MODEL_FILE = "Mistral-7B-Instruct-v0.3.Q4_K_M.gguf"
MODEL_URL = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3.Q4_K_M.gguf"

if not os.path.exists(MODEL_FILE):
    print(f"📥 Downloading {MODEL_FILE}...")
    print("⏱️  This will take about 2 minutes\n")
    !wget -q --show-progress {MODEL_URL}
    print(f"\n✅ Model downloaded! Size: {os.path.getsize(MODEL_FILE) / 1024**3:.2f} GB")
else:
    print(f"✅ Model already exists! Size: {os.path.getsize(MODEL_FILE) / 1024**3:.2f} GB")

print("\n💡 Model is ready for GPU inference!")

## Step 4: Setup Pinecone Connection

In [ ]:
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings

print("🔗 Connecting to Pinecone...")

PINECONE_API_KEY = "pcsk_5A9JjS_JVvYF7aE1kieuSnTXitm1pEMdVhg2wkpijQ3hiV9aC7rZ2CurG5qRfXE9FxHLAh"
INDEX_NAME = "mindnex-responses"

try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    pinecone_index = pc.Index(INDEX_NAME)
    
    # Load embeddings on GPU
    print("🔤 Loading embedding model on GPU...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'}
    )
    
    # Check stats
    stats = pinecone_index.describe_index_stats()
    print(f"\n✅ Pinecone connected!")
    print(f"✅ Vectors stored: {stats['total_vector_count']}")
    print(f"✅ Embeddings on GPU: CUDA")
    
except Exception as e:
    print(f"⚠️  Pinecone error: {e}")
    pinecone_index = None
    embeddings = None

## Step 5: Load Model with GPU Acceleration

In [ ]:
from llama_cpp import Llama
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from datetime import datetime

print("=" * 60)
print("🚀 Loading Mistral-7B with FREE GPU")
print("=" * 60)

llm = LlamaCpp(
    model_path="Mistral-7B-Instruct-v0.3.Q4_K_M.gguf",
    n_ctx=8192,  # Large context
    n_threads=2,
    n_gpu_layers=-1,  # ALL layers on GPU
    n_batch=512,
    temperature=0.7,
    top_p=0.95,
    repeat_penalty=1.2,
    max_tokens=500,
    verbose=False
)

print("\n✅ Model loaded on GPU!")
print("✅ Using ALL model layers on T4 GPU")
print("✅ Expected speed: 40-60 tokens/sec (10x faster than Mac!)\n")

# Create prompt template
prompt_template = PromptTemplate(
    input_variables=["topic", "age"],
    template="[INST] Explain {topic} in detail for a {age} year old to understand. [/INST]"
)

chain = prompt_template | llm

print("=" * 60)

## Step 6: Define Helper Functions

In [ ]:
def store_in_pinecone(topic: str, response: str, age: str):
    """Store response in Pinecone with GPU-accelerated embeddings"""
    if not pinecone_index or not embeddings:
        return None
    
    try:
        print("\n💾 Storing in Pinecone...")
        
        # Generate embedding on GPU
        embedding = embeddings.embed_query(response)
        
        # Create unique ID
        vector_id = f"response_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{hash(topic) % 10000}"
        
        # Metadata
        metadata = {
            'topic': topic,
            'age': int(age) if age.isdigit() else 12,
            'response': response,
            'word_count': len(response.split()),
            'character_count': len(response),
            'timestamp': datetime.now().isoformat(),
            'response_preview': response[:200],
            'device': 'cuda',
            'gpu_model': torch.cuda.get_device_name(0),
            'platform': 'Google Colab Free'
        }
        
        # Store in Pinecone
        pinecone_index.upsert(
            vectors=[{
                'id': vector_id,
                'values': embedding,
                'metadata': metadata
            }]
        )
        
        print(f"   ✅ Stored in Pinecone: {vector_id}")
        print(f"   ✅ Embedding generated on GPU")
        return vector_id
    
    except Exception as e:
        print(f"   ⚠️  Error: {e}")
        return None


def generate_text(topic: str, age: str) -> tuple:
    """Generate text with GPU acceleration and timing"""
    try:
        print(f"\n🚀 Generating response for: {topic}")
        print(f"👤 Target age: {age}")
        
        # Time the generation
        start_time = datetime.now()
        
        response = chain.invoke({"topic": topic, "age": age})
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        tokens = len(response.split())
        tokens_per_sec = tokens / duration if duration > 0 else 0
        
        print(f"\n⚡ Stats:")
        print(f"   • Tokens: {tokens}")
        print(f"   • Time: {duration:.2f}s")
        print(f"   • Speed: {tokens_per_sec:.1f} tokens/sec")
        print(f"   • GPU: {torch.cuda.get_device_name(0)}")
        
        # Store in Pinecone
        if pinecone_index and embeddings:
            store_in_pinecone(topic, response, age)
        
        return response, tokens_per_sec
        
    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"Error: {str(e)}", 0

print("✅ Helper functions defined!")

## Step 7: Run Benchmark Test

In [ ]:
print("=" * 60)
print("🏃 Running GPU Performance Benchmark")
print("=" * 60)

# Test with Machine Learning topic
response, speed = generate_text("Machine Learning", "25")

print("\n" + "=" * 60)
print("📊 BENCHMARK RESULTS")
print("=" * 60)
print(f"\nGPU: {torch.cuda.get_device_name(0)}")
print(f"Speed: {speed:.1f} tokens/sec")
print(f"\n🎉 This is ~10x FASTER than your Mac!")
print(f"🆓 And it's completely FREE!")
print("\n" + "=" * 60)

print("\n--- GENERATED TEXT ---\n")
print(response)

## Step 8: Interactive Mode - Test Your Own Topics!

In [ ]:
# Interactive testing
print("=" * 60)
print("💬 Interactive Mode - Test FREE GPU!")
print("=" * 60)
print("\n💡 Try different topics to test the speed!")
print("Examples: Neural Networks, Quantum Physics, Blockchain\n")

topic = input("Enter topic: ")
age = input("Enter age: ")

response, speed = generate_text(topic, age)

print("\n" + "=" * 60)
print("📝 RESULT")
print("=" * 60)
print(f"\nSpeed: {speed:.1f} tokens/sec on FREE GPU")
print(f"Mac would take: {speed * 10:.0f}s (vs {len(response.split()) / speed:.0f}s on GPU)")
print("\n--- RESPONSE ---\n")
print(response)

if pinecone_index:
    stats = pinecone_index.describe_index_stats()
    print(f"\n✅ Total vectors stored: {stats['total_vector_count']}")

## Step 9: Check GPU Usage

In [ ]:
print("=" * 60)
print("📊 GPU Usage Statistics")
print("=" * 60)

if torch.cuda.is_available():
    print(f"\nGPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
    print(f"\n🔥 Using FREE T4 GPU - No cost!")
    print(f"⏱️  Session time remaining: Check top-right corner")
    
    # Show nvidia-smi
    print("\n" + "=" * 60)
    print("NVIDIA GPU Info:")
    print("=" * 60)
    !nvidia-smi
else:
    print("\n❌ GPU not available")

## 💡 Tips for Free Tier:

### Session Limits:
- **Free Tier:** 12 hours max per session
- **GPU Time:** Limited to a few hours per day
- **Auto-disconnect:** After 90 minutes of inactivity

### Maximizing Free GPU Time:
1. Keep browser tab active (prevents disconnect)
2. Run interactive cell periodically
3. Download important results before session ends
4. Re-run setup cells when reconnecting

### Upgrade Options:
- **Colab Pro:** $10/month
  - Longer sessions (24 hours)
  - Priority GPU access
  - More GPU time
  - Background execution

### Performance Comparison:
| Platform | Speed | Cost | Temperature |
|----------|-------|------|-------------|
| Mac M1 | 5-10 tok/s | $0 | 80-90°C |
| Colab Free | 40-60 tok/s | $0 | 60°C |
| Colab Pro | 40-60 tok/s | $10/mo | 60°C |

---

## 🎉 You're Running on FREE GPU!

**Benefits:**
- ✅ No Mac overheating
- ✅ 10x faster than Mac
- ✅ Completely free
- ✅ No installation needed
- ✅ Access from any browser
- ✅ All data saved to Pinecone

**Perfect for:**
- Testing Phase 1 data collection
- Rapid prototyping
- Cost-free development
- Learning GPU optimization